# 🐸 Bombina Fine-Tuning on Google Colab

This notebook fine-tunes Qwen2.5-Coder-3B on your pentest reasoning dataset.

**Requirements:**
- Google Colab with T4 GPU (free tier works)
- Upload `train.jsonl` from your Bombina project

**Output:** LoRA adapter (~300MB) to download and use locally

In [ ]:
# Step 1: Install dependencies
!pip install -q unsloth
!pip install -q --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --upgrade torch
!pip install -q xformers trl peft accelerate bitsandbytes triton

In [ ]:
# Step 2: Check GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Step 3: Upload your training data
from google.colab import files
print("Upload train.jsonl from your Bombina/data/ folder:")
uploaded = files.upload()

In [ ]:
# Step 4: Load model with Unsloth (4-bit quantization)
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen2.5-Coder-3B-Instruct",
    max_seq_length=2048,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

print("✅ Model loaded successfully!")

In [ ]:
# Step 5: Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("✅ LoRA adapters added!")

In [ ]:
# Step 6: Prepare dataset
from datasets import load_dataset

# Load your uploaded training data
dataset = load_dataset('json', data_files='train.jsonl', split='train')
print(f"✅ Loaded {len(dataset)} training samples")

# Format for instruction tuning
def format_prompt(example):
    instruction = example.get('instruction', '')
    input_text = example.get('input', '')
    output = example.get('output', '')
    
    if input_text:
        text = f"""### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        text = f"""### Instruction:
{instruction}

### Response:
{output}"""
    
    return {"text": text}

dataset = dataset.map(format_prompt)
print("✅ Dataset formatted!")
print(f"\nSample:\n{dataset[0]['text'][:500]}...")

In [ ]:
# Step 7: Training configuration
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=42,
        output_dir="outputs",
        save_strategy="epoch",
    ),
)

print("✅ Trainer configured!")

In [ ]:
# Step 8: TRAIN! (takes 1-3 hours on T4)
print("🚀 Starting training...")
print("This will take 1-3 hours on a T4 GPU.\n")

trainer_stats = trainer.train()

print("\n✅ Training complete!")
print(f"Training time: {trainer_stats.metrics['train_runtime']/60:.1f} minutes")

In [ ]:
# Step 9: Save LoRA adapter
model.save_pretrained("bombina_lora")
tokenizer.save_pretrained("bombina_lora")

print("✅ LoRA adapter saved to 'bombina_lora/'")

# Check size
!du -sh bombina_lora/

In [ ]:
# Step 10: Test the fine-tuned model
FastLanguageModel.for_inference(model)

test_prompt = """### Instruction:
You are conducting an internal pentest against a hardened Active Directory environment.

### Input:
EDR is deployed (CrowdStrike), SMB signing enabled, NTLM restricted. You have a standard domain user account.

### Response:
"""

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.3)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Test Response:")
print(response.split("### Response:")[1] if "### Response:" in response else response)

In [ ]:
# Step 11: Export to GGUF for Ollama
print("Converting to GGUF format for Ollama...")

# First merge LoRA with base model
model.save_pretrained_merged("bombina_merged", tokenizer, save_method="merged_16bit")

# Install llama.cpp for conversion
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && make -j

# Convert to GGUF
!python llama.cpp/convert_hf_to_gguf.py bombina_merged --outfile bombina-finetuned.gguf --outtype q4_k_m

print("\n✅ GGUF file created!")
!ls -lh bombina-finetuned.gguf

In [ ]:
# Step 12: Download the fine-tuned model
from google.colab import files

# Zip the LoRA adapter
!zip -r bombina_lora.zip bombina_lora/

print("\n📥 Downloading files...")
print("\n1. LoRA adapter (for merging later):")
files.download('bombina_lora.zip')

print("\n2. GGUF file (ready for Ollama):")
files.download('bombina-finetuned.gguf')

## 🎉 Done!

### On your Kali machine:

```bash
# 1. Create Modelfile
echo 'FROM ./bombina-finetuned.gguf' > Modelfile.trained
echo 'PARAMETER temperature 0.3' >> Modelfile.trained
echo 'PARAMETER num_ctx 2048' >> Modelfile.trained

# 2. Import to Ollama
ollama create bombina-trained -f Modelfile.trained

# 3. Test
ollama run bombina-trained

# 4. Run evaluation
python evaluation/scenarios.py --model bombina-trained
```